# Read Before Starting.

- The cells should be executed in order.
- To access all parts of the code, simply click on the arrowheads to the left of the titles or on the gray boxes labeled " +... cells hidden ".
- If any informational messages/warnings appear (without affecting the subsequent process), this will be noted in the relevant cell.
- For cells that require modification, a comment "# - NECESSARY MODIFICATIONS - " will be present at the beginning of the cell (with a detailed explanation of the affected lines of code).
- In order to avoid cluttering the cells, the details of each line of code are not explained, but a general explanatory comment is provided at the beginning of each cell.
- Before starting, it is recommended to place all the necessary images for the processing (light frames, bias frames, dark frames, flat frames) into the same folder.
- Before starting, save this notebook in the same folder as the one containing your raw image folder.

<h1 style="text-align: center; font-weight: bold;">CODE</h1>


In [ ]:
# Libraries useful for this code

import numpy as np

import matplotlib.pyplot as plt

import shutil

from astropy.io import fits
from astropy import units as u
from astropy.nddata import CCDData
from astropy.stats import mad_std
from astropy.stats import sigma_clip
from astropy.io.fits import Header

from pathlib import Path

from ccdproc import ImageFileCollection
import ccdproc as ccdp

In [ ]:
# - NECESSARY MODIFICATIONS 

# Folder path for image collection to be processed (light frame, bias frame etc.)
raw_path = Path('Images to process') #replace “Images to process” with the name of your folder
raw_data = ccdp.ImageFileCollection(raw_path)

# Create a folder (in the same place) for images currently being processed/processed
calibrated_path = Path('Image processing') # if necessary, replace “Image processing” with another name as required
calibrated_path.mkdir(exist_ok=True) 
reduced_data = ccdp.ImageFileCollection(calibrated_path)

# Warnings will appear.

# Information on the contents of the “raw” images folder

In [ ]:
# Viewing the contents of the source folder.

raw_data.summary

In [ ]:
# Counting the different frame types available in the source folder.

dark_count = len(list(raw_data.hdus(imagetyp='Dark Frame')))
bias_count = len(list(raw_data.hdus(imagetyp='Bias Frame')))
flat_count = len(list(raw_data.hdus(imagetyp='Flat Field')))
light_count = len(list(raw_data.hdus(imagetyp='Light Frame')))

print(f"Number of dark frames : {dark_count}")
print(f"Number of bias frames : {bias_count}")
print(f"Number of flat frames : {flat_count}")
print(f"Number of light frames : {light_count}")

if dark_count == 0:
    print("Warning: No dark frames found in the source folder!")
if bias_count == 0:
    print("Warning: No bias frames found in the source folder!")
if flat_count == 0:
    print("Warning: No flat frames found in the source folder!")
if light_count == 0:
    print("Warning: No light frames found in the source folder!")

# Calibration Frames

## Bias Frame

In [ ]:
# Copy Bias Frame to current image folder

biases = raw_data.files_filtered(imagetyp='Bias Frame', include_path=True)

for bias in biases:
    shutil.copy(bias, calibrated_path)

reduced_data.refresh()

In [ ]:
# Definition of “bias frames” in the images-in-progress folder

calibrated_biases = reduced_data.files_filtered(imagetyp='Bias Frame', include_path=True)

In [ ]:
# Combining bias frames into a Master Bias Frame (method = average) and Master Flat Frame registration.

total_sum = None

for bias_file in calibrated_biases:
    data = fits.getdata(bias_file)
    data = data.astype(np.float64) 
    if total_sum is None:
        total_sum = data
    else:
        total_sum += data

average_bias = total_sum / len(calibrated_biases)

header = fits.getheader(calibrated_biases[0])
header['combined'] = True

combined_bias_file_name = 'Master Bias Frame.fit'
fits.writeto(calibrated_path / combined_bias_file_name, average_bias.astype(np.float32), header=header, overwrite=True)

reduced_data.refresh()

In [ ]:
# Definition of the Bias Frame master in the images-in-progress folder.

combined_bias = CCDData.read(reduced_data.files_filtered(imagetyp='Bias Frame',combined=True,include_path=True)[0],unit='adu') 

# Warnings will appear.

## Dark Frame

In [ ]:
# Subtract the bias frame for each dark frame and store these dark frames in the images in process folder.

for ccd, file_name in raw_data.ccds(imagetyp='Dark Frame',ccd_kwargs={'unit': 'adu'},return_fname=True):
    
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    ccd.write(calibrated_path / file_name,overwrite=True)

reduced_data.refresh()

# Warnings will appear.

In [ ]:
# Defining what “dark frames” are in the images-in-progress folder.

calibrated_darks = reduced_data.files_filtered(imagetyp='Dark Frame', include_path=True)

In [ ]:
# Dark frame combination (method = average) and Master Dark Frame registration.

total_sum = None  

for dark_file in calibrated_darks:
    data = fits.getdata(dark_file) 
    if total_sum is None:
        total_sum = data  
    else:
        total_sum += data  

header = fits.getheader(calibrated_darks[0])
header['combined'] = True

average_dark = total_sum / len(calibrated_darks)

dark_file_name = 'Master Dark Frame.fit'
fits.writeto(calibrated_path / dark_file_name, average_dark, header=header, overwrite=True)

reduced_data.refresh()

In [ ]:
# Defining the Master Dark Frame in the images-in-progress folder.

combined_dark= CCDData.read(reduced_data.files_filtered(imagetyp='Dark Frame', combined=True, include_path=True)[0],unit='adu')

# Information will appear.
# Warnings will appear.

## Flat Frame

In [ ]:
# Subtracting Master Bias Frame, Master Dark Frame (aligned with the exposure time of the respective flat frame), 
# Normalizing (method = average) the flat frames, then saving these flat frames in the images-in-progress folder.

dark_exposure_time = combined_dark.header['exptime'] * u.second

for flat_frame, file_name in raw_data.ccds(imagetyp='Flat Field', ccd_kwargs={'unit': 'adu'}, return_fname=True):
    
    flat_reduced = ccdp.subtract_bias(flat_frame, combined_bias)

    flat_exposure_time = flat_reduced.header['exptime'] * u.second
    scaled_dark = combined_dark.multiply(flat_exposure_time / dark_exposure_time)
    scaled_dark.header['exptime'] = flat_exposure_time.value
    flat_frame_reduced = ccdp.subtract_dark(flat_reduced, 
                                            scaled_dark, 
                                            exposure_time='exptime', 
                                            exposure_unit=u.second)

    max_pixel_value = np.max(flat_frame_reduced.data) 
    if max_pixel_value > 0: 
        flat_frame_reduced.data /= max_pixel_value  

    reduced_data_file_path = calibrated_path / Path(file_name).name  

    flat_frame_reduced.write(reduced_data_file_path, overwrite=True)

reduced_data.refresh()

# Warnings will appear.

In [ ]:
# Checking the filters used for flat frames.

flat_filters = set(h['filter'] for h in reduced_data.headers(imagetyp='Flat Field'))
flat_filters

In [ ]:
# Combining flat frames (method = average) by filter and registering the various Master Flat Frames.

for filt in flat_filters:
    to_combine = reduced_data.files_filtered(imagetyp='Flat Field', filter=filt, include_path=True)

    total_sum = sum(fits.getdata(flat_file) for flat_file in to_combine)
    average_flat = total_sum / len(to_combine)

    header = fits.getheader(to_combine[0])
    header['combined'] = True

    combined_flat_file_name = f'Master Flat Frame filter {filt}.fit'  

    fits.writeto(calibrated_path / combined_flat_file_name, average_flat, header=header, overwrite=True)

reduced_data.refresh()

# Light Frame

## Calibration

In [ ]:
# Creation of a dictionary (useful when multiple filters and exposure times are involved) for subsequent data processing.

combined_darks = {ccd.header['exptime']: ccd for ccd in reduced_data.ccds(imagetyp='Dark Frame',ccd_kwargs={'unit': u.adu}, combined=True)}
combined_flats = {ccd.header['filter']: ccd for ccd in reduced_data.ccds(imagetyp='Flat Field', ccd_kwargs={'unit': u.adu}, combined=True)}
combined_bias = [ccd for ccd in reduced_data.ccds(imagetyp='Bias Frame', ccd_kwargs={'unit': u.adu}, combined=True)][0]

# Information will appear.
# Warnings will appear.

In [ ]:
# Subtraction of the Master Bias Frame and Master Dark Frame (aligned with the exposure time of each light frame), 
# followed by division by the Master Flat Frame. Saving the processed light frames in the 'images-in-progress' folder.

all_reds = []
light_ccds = []

epsilon = 1e-6

for light, file_name in raw_data.ccds(imagetyp='Light Frame', return_fname=True, ccd_kwargs=dict(unit='adu')):
    light_ccds.append(light)
    
    original_header = light.header.copy()

    light_reduced = ccdp.subtract_bias(light, combined_bias)
    
    light_exposure_time = light_reduced.header['exptime'] * u.second
    scaled_dark = combined_dark.multiply(light_exposure_time / dark_exposure_time)
    scaled_dark.header['exptime'] = light_exposure_time.value
    light_frame_reduced = ccdp.subtract_dark(light_reduced,scaled_dark,exposure_time='exptime',exposure_unit=u.second)
    
    current_filter = light_frame_reduced.header['filter']
    if current_filter in combined_flats:
        good_flat = combined_flats[current_filter]
        good_flat.data = np.where(good_flat.data == 0, epsilon, good_flat.data)
        light_frame_reduced = ccdp.flat_correct(light_frame_reduced, good_flat)
    else:
        print(f"No flat frame available for filter {current_filter}. The image  {file_name} will not be calibrated.")
        continue  
    
    light_frame_reduced.header.update(original_header)
    
    all_reds.append(light_frame_reduced)
    light_frame_reduced.write(calibrated_path / file_name, overwrite=True)

reduced_data.refresh()

# Information will appear.
# Warnings will appear.

## Combination

In [ ]:
# Combination of the images with tracking of the brightest image shape and image registration.

from scipy.ndimage import shift
from skimage import measure

def replace_out_of_range_pixels(image, min_threshold=0, max_threshold=65535):
    result = image.copy()
    out_of_range_coords = np.argwhere((result < min_threshold) | (result > max_threshold))
    for y, x in out_of_range_coords:
        neighbors = result[max(0, y-1):y+2, max(0, x-1):x+2]
        valid_neighbors = neighbors[(neighbors >= min_threshold) & (neighbors <= max_threshold)]
        if valid_neighbors.size > 0:  
            result[y, x] = valid_neighbors.mean()
        else:
            result[y, x] = min_threshold if result[y, x] < min_threshold else max_threshold
    
    return result

light_filters = set(h['filter'] for h in reduced_data.headers(imagetyp='Light Frame'))

for filt in light_filters:
    to_combine = reduced_data.files_filtered(imagetyp='Light Frame', filter=filt, include_path=True)
    light_data = []  

    reference_image = fits.getdata(to_combine[0])
    light_data.append(reference_image)

    def find_object_center(image):
        threshold_value = np.percentile(image, 90)  # Ajuster si nécessaire
        thresholded_image = image > threshold_value
        labeled_image = measure.label(thresholded_image)
        regions = measure.regionprops(labeled_image)
        if not regions:
            return None  
        largest_region = max(regions, key=lambda r: r.area)
        return largest_region.centroid

    y_ref, x_ref = find_object_center(reference_image)
    if y_ref is None or x_ref is None:
        print(f"No region found in the reference image for the filter {filt}.")
        continue

    for light_file in to_combine[1:]:
        data = fits.getdata(light_file)
        y_curr, x_curr = find_object_center(data)

        if y_curr is None or x_curr is None:
            print(f"No region found in image {light_file}  for filter {filt}.")
            continue

        y_shift = int(round(y_ref - y_curr))
        x_shift = int(round(x_ref - x_curr))
        aligned_image = shift(data, shift=(y_shift, x_shift))

        light_data.append(aligned_image)

    combined_light = np.mean(light_data, axis=0)

    combined_light = replace_out_of_range_pixels(combined_light)

    header = fits.getheader(to_combine[0])
    header['combined'] = True
    combined_light_file_name = f'Combined_light_filter_{filt}_(tracking).fit'
    fits.writeto(calibrated_path / combined_light_file_name, combined_light, header=header, overwrite=True)

reduced_data.refresh()

# Image visualization

## Black and white

In [ ]:
# - NECESSARY MODIFICATIONS 

# Displays and saves the cropped and calibrated images.

selected_filter = 'B'  # Filter example, replace with 'R', 'V', 'L', etc.

combined_light_file_name = f'Combined_light_filter_{selected_filter}_(tracking).fit'
combined_light_path = calibrated_path / combined_light_file_name
combined_image = fits.getdata(combined_light_path)

crop_margin = 20  # Adjust this number to define how many pixels to cut from the edges (optional)
if crop_margin > 0:
    combined_image = combined_image[crop_margin:-crop_margin, crop_margin:-crop_margin]

output_file_name = f'Combined_light_grey_cropped_{selected_filter}.fit'
fits.writeto(output_file_name, combined_image, overwrite=True)
print(f"Image saved in black and white in FITS format: {output_file_name}")

vmin, vmax = combined_image.min(), combined_image.max()
plt.figure(figsize=(8, 8))
img = plt.imshow(combined_image, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)  
plt.title(f'Combined image (min-max scale, filter {selected_filter})')
plt.axis('off')

cbar = plt.colorbar(img, orientation='vertical', fraction=0.05, pad=0.04, shrink=0.6)  
cbar.set_label('Pixel Value')

plt.tight_layout()
plt.show()

## Colors and RGB

In [ ]:
# - NECESSARY MODIFICATIONS 

# Visualization of processed, cropped, and color-applied images, and saving them in the notebook folder.

from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

def apply_color_map_2(image, filter_name):
    color_map_dict = {
        'R': 'Reds',
        'V': 'Greens',
        'B': 'Blues',
        'Halpha': 'YlOrRd',
        'L': 'gray',
        'I': 'hot',       
        'U': 'plasma'    
    }
    color_map = color_map_dict.get(filter_name, 'viridis')  
    
    norm_image = (image - image.min()) / (image.max() - image.min())
    
    colored_image = cm.get_cmap(color_map)(norm_image)[..., :3]  
    return colored_image

selected_filter = 'R'  # Filter example, replace with 'R', 'V', 'L', etc.

combined_light_file_name = f'Combined_light_filter_{selected_filter}_(tracking).fit' 
combined_light_path = calibrated_path / combined_light_file_name
combined_image = fits.getdata(combined_light_path)

colored_combined_image = apply_color_map_2(combined_image, selected_filter)

crop_margin = 20  # Adjust this number to define how many pixels to cut from the edges (optional)
if crop_margin > 0:
    colored_combined_image = colored_combined_image[crop_margin:-crop_margin, crop_margin:-crop_margin]

output_file_name = f'Combined_light_colored_{selected_filter}.png'
plt.imsave(output_file_name, colored_combined_image)

print(f"Registered image : {output_file_name}")

plt.figure(figsize=(8, 8))
plt.imshow(colored_combined_image, origin='lower')
plt.title(f'Combined color image (filter {selected_filter})') 
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# - NECESSARY MODIFICATIONS 

# Alternative colorization method (works well for the Sun, for example).
# Visualization of processed, cropped, and color-applied images, and saving them in the notebook folder.


def apply_color_map(image, selected_filter, lower_percentile=5, upper_percentile=95):
    vmin = np.percentile(image, lower_percentile)
    vmax = np.percentile(image, upper_percentile)
    image_normalized = np.clip((image - vmin) / (vmax - vmin), 0, 1)

    if selected_filter == 'U':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 2] = image_normalized * 0.9  
        colored_image[..., 0] = image_normalized * 0.4  

    elif selected_filter == 'B':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 2] = image_normalized  

    elif selected_filter == 'V':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 0] = image_normalized * 0.6  
        colored_image[..., 1] = image_normalized * 0.9  
        colored_image[..., 2] = image_normalized * 0.7  

    elif selected_filter == 'R':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 0] = image_normalized * 0.8  
        colored_image[..., 1] = image_normalized * 0.3  

    elif selected_filter == 'I':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 0] = image_normalized * 0.9  
        colored_image[..., 1] = image_normalized * 0.1  

    elif selected_filter == 'Halpha':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 0] = image_normalized * 0.8 
        colored_image[..., 1] = image_normalized * 0.5  

    elif selected_filter == 'L':
        colored_image = np.zeros((*image.shape, 3))
        colored_image[..., 0] = image_normalized * 0.5  
        colored_image[..., 1] = image_normalized * 0.55  
        colored_image[..., 2] = image_normalized * 0.5  

    else:
        raise ValueError(f"Unknown filter: {selected_filter}")

    colored_image[image == 0] = 0 

    return colored_image

selected_filter = 'B'  # Filter example, replace with 'R', 'V', 'L', etc.

combined_light_file_name = f'Combined_light_filter_{selected_filter}_(tracking).fit'
combined_light_path = calibrated_path / combined_light_file_name
combined_image = fits.getdata(combined_light_path)

colored_combined_image = apply_color_map(combined_image, selected_filter)


crop_margin = 20  # Adjust this number to define how many pixels to cut from the edges (optional)
if crop_margin > 0:
    colored_combined_image = colored_combined_image[crop_margin:-crop_margin, crop_margin:-crop_margin]

output_file_name = f'combined_light_colored_{selected_filter}.png' 
plt.imsave(output_file_name, colored_combined_image) 
print(f"Registered image : {output_file_name}")

plt.figure(figsize=(8, 8))
plt.imshow(colored_combined_image, origin='lower')
plt.title(f'Combined color image (filter {selected_filter})')
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# - NECESSARY MODIFICATIONS 

# Visualization of processed, cropped, and RGB color-applied images, and saving them in the notebook folder.

from skimage import measure
from scipy.ndimage import shift
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from skimage import img_as_ubyte
from skimage.io import imsave

def read_fits(file):
    with fits.open(file) as hdul:
        data = hdul[0].data
        exposure_time = hdul[0].header.get('EXPTIME', 1)
    data[data < 0] = 0
    data -= np.min(data)
    data /= np.max(data)
    return data, exposure_time

def find_object_center(image):
    threshold_value = np.percentile(image, 70) # Change percentile value if images are not aligned properly (optional)
    thresholded_image = image > threshold_value
    labeled_image = measure.label(thresholded_image)
    largest_region = max(measure.regionprops(labeled_image), key=lambda r: r.area)
    return largest_region.centroid

blue_file = '/users/my_combined_image_blue_filter' # Modify the paths of combined images according to the B filters.
green_file = '/users/my_combined_image_green_filter' # Modify the paths of combined images according to the V filters.
red_file = '/users/my_combined_image_red_filter' # Modify the paths of combined images according to the R filters.
red_data, red_exposure = read_fits(red_file)
green_data, green_exposure = read_fits(green_file)
blue_data, blue_exposure = read_fits(blue_file)

min_exposure = min(red_exposure, green_exposure, blue_exposure)
red_data *= min_exposure / red_exposure
green_data *= min_exposure / green_exposure
blue_data *= min_exposure / blue_exposure

y_ref, x_ref = find_object_center(red_data)
y_green, x_green = find_object_center(green_data)
y_blue, x_blue = find_object_center(blue_data)

aligned_green = shift(green_data, shift=(y_ref - y_green, x_ref - x_green))
aligned_blue = shift(blue_data, shift=(y_ref - y_blue, x_ref - x_blue))
aligned_red = red_data

# Adjust intensities to enhance colors (optional)
aligned_red *= 3
aligned_green *= 2.1
aligned_blue *= 0.2

crop_margin = 20  # Adjust this number to define how many pixels to cut from the edges (optional)
aligned_red_cropped = aligned_red[crop_margin:-crop_margin, crop_margin:-crop_margin]
aligned_green_cropped = aligned_green[crop_margin:-crop_margin, crop_margin:-crop_margin]
aligned_blue_cropped = aligned_blue[crop_margin:-crop_margin, crop_margin:-crop_margin]

aligned_red_cropped = np.clip(aligned_red_cropped, 0, 1)
aligned_green_cropped = np.clip(aligned_green_cropped, 0, 1)
aligned_blue_cropped = np.clip(aligned_blue_cropped, 0, 1)

rgb_image = np.dstack((aligned_red_cropped, aligned_green_cropped, aligned_blue_cropped))

output_file_name_png = 'Combined_light_RGB.png'
imsave(output_file_name_png, img_as_ubyte(rgb_image))
print(f"RGB image saved in PNG format: {output_file_name_png}")

plt.figure(figsize=(10, 10))
plt.imshow(rgb_image, origin='lower')
plt.xlabel('X Pixels')
plt.ylabel('Y Pixels')
plt.title('Image RGB')
plt.axis('off')
plt.show()